In [1]:
from pyspark import SparkContext
sc = SparkContext(master = 'local[*]', appName = 'HomeWork_03')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 14:05:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
book_rdd = sc.textFile("./BooksRating-CSV/Books.csv")
user_rdd = sc.textFile("./BooksRating-CSV/Users.csv")
rate_rdd = sc.textFile("./BooksRating-CSV/Book-Ratings.csv")

In [3]:
print(f"book Count: {book_rdd.count()}, User Count: {user_rdd.count()}, Rate Count: {rate_rdd.count()}")

book Count: 271380, User Count: 278699, Rate Count: 1149781


In [4]:
pair_rate_rdd = rate_rdd.map(lambda x: (x.split(';')[1].replace('"', ''), x.split(';')[2].replace('"', '')))
header = pair_rate_rdd.first()
pair_rate_rdd = pair_rate_rdd.filter(lambda x: x != header)
pair_rate_rdd.take(5)

[('034545104X', '0'),
 ('0155061224', '5'),
 ('0446520802', '0'),
 ('052165615X', '3'),
 ('0521795028', '6')]

In [5]:
key_value_count = pair_rate_rdd.map(lambda x: (x[0], (int(x[1]), 1)))
sum_count = key_value_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
sum_count.take(5)

[('034545104X', (176, 60)),
 ('0155061224', (5, 2)),
 ('0521795028', (6, 1)),
 ('0600570967', (6, 1)),
 ('342310538', (10, 2))]

In [6]:
mean_rdd = sum_count.map(lambda x: (x[0], x[1][0] / x[1][1]))
mean_rdd.take(5)

[('034545104X', 2.933333333333333),
 ('0155061224', 2.5),
 ('0521795028', 6.0),
 ('0600570967', 6.0),
 ('342310538', 5.0)]

In [7]:
book_title_rdd = book_rdd.map(lambda x: (x.split(';')[0].replace('"', ''), x.split(';')[1].replace('"', '')))
book_title_rdd.take(5)

[('ISBN', 'BookTitle'),
 ('0195153448', 'Classical Mythology'),
 ('0002005018', 'Clara Callan'),
 ('0060973129', 'Decision in Normandy'),
 ('0374157065',
  'Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It')]

In [8]:
pair_user_rdd = user_rdd.map(lambda x: (x.split(';')[0].replace('"', ''), x.split(';')[1].replace('"', '')))
pair_user_rdd.take(5)

[('UserID', 'USERNAME'),
 ('1', 'bzsufoRTLN2'),
 ('2', 'fq7kfHg4VEI'),
 ('3', 'W0Hbkd3xR8v'),
 ('4', 'W51GahAx5Ap')]

In [9]:
pair_rate_rdd = rate_rdd.map(lambda x: (x.split(';')[0].replace('"', ''), (x.split(';')[1].replace('"', ''), x.split(';')[2].replace('"', ''))))
pair_rate_rdd.take(5)

[('userid', ('isbn', 'rate')),
 ('276725', ('034545104X', '0')),
 ('276726', ('0155061224', '5')),
 ('276727', ('0446520802', '0')),
 ('276729', ('052165615X', '3'))]

In [10]:
join_rate_rdd = pair_rate_rdd.leftOuterJoin(pair_user_rdd)
join_rate_rdd.take(5)

[('276725', (('034545104X', '0'), 'qP0KgArgnkK')),
 ('276727', (('0446520802', '0'), 'AWJdCVnDLd8')),
 ('276744', (('038550120X', '7'), 'CoM54wt6AiU')),
 ('276746', (('0425115801', '0'), '7uxi3eD6COS')),
 ('276746', (('0449006522', '0'), '7uxi3eD6COS'))]

In [11]:
join_rate_rdd = join_rate_rdd.map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))
join_rate_rdd.take(5)

[('034545104X', ('qP0KgArgnkK', '0')),
 ('0446520802', ('AWJdCVnDLd8', '0')),
 ('038550120X', ('CoM54wt6AiU', '7')),
 ('0425115801', ('7uxi3eD6COS', '0')),
 ('0449006522', ('7uxi3eD6COS', '0'))]

In [12]:
join_rate_title_rdd = join_rate_rdd.leftOuterJoin(book_title_rdd)
join_rate_title_rdd.take(5)

[('0446520802', (('AWJdCVnDLd8', '0'), 'The Notebook')),
 ('0446520802', (('qsTAPT3KK0R', '0'), 'The Notebook')),
 ('0446520802', (('eSB2qb78rCs', '0'), 'The Notebook')),
 ('0446520802', (('eJeGfMDh4PC', '6'), 'The Notebook')),
 ('0446520802', (('DbpWkW8ft4x', '0'), 'The Notebook'))]

In [19]:
final_join_rdd = join_rate_title_rdd.leftOuterJoin(mean_rdd)
final_join_rdd.count()

1149781

In [20]:
final_join_rdd = final_join_rdd.map(lambda x: (x[1][0][0][0], x[1][0][1], x[1][0][0][1], x[1][1]))
final_join_rdd.takeSample(5, 5)

[('IzXx1URtubx', 'Demolition Angel', '6', 3.0476190476190474),
 ('r5ygrhe2D3u',
  'Echo Burning (Jack Reacher Novels (Paperback))',
  '0',
  4.458333333333333),
 ('0DR1MSX3e2Y',
  'Objects of Desire: The Lives of Antiques and Those Who Pursue Them',
  '10',
  10.0),
 ('MJrEKBEpuYd',
  "Wrangler'S Wedding (Silhouette Romance, No 1149)",
  '0',
  5.333333333333333),
 ('Hnq18Bclvb4', 'The Notebook', '0', 3.4064516129032256)]